In [ ]:
which qvina-w

### Setup

In [ ]:
exhaustiveness=400
cpus=6
ligand_name="AcetylCoA"
pdbs_list="pdbs.list"

### Generate configs

In [ ]:
ligand=$(dir -1 ligands | grep "$ligand_name.pdbqt")

In [ ]:
mkdir -p configs
while read pdb_filename
do
pdb=$(basename -s .pdbqt $pdb_filename)
rm -f ./configs/$pdb.conf

# Read grid
center=$(cat ./grids/grid_$pdb.txt | grep center | tr -s ' ' '\t')
center_x=$(cut -f 2 <<< "$center")
center_y=$(cut -f 3 <<< "$center")
center_z=$(cut -f 4 <<< "$center")
size=$(cat ./grids/grid_$pdb.txt | grep npts | tr -s ' ' '\t')
size_x=$(cut -f 2 <<< "$size")
size_y=$(cut -f 3 <<< "$size")
size_z=$(cut -f 4 <<< "$size")

# Generate config file
cat > ./configs/$pdb.conf << EOL
receptor = ./pdbs/$pdb.pdbqt
ligand = ./ligands/$ligand

center_x = $center_x
center_y = $center_y
center_z = $center_z

size_x = $size_x
size_y = $size_y
size_z = $size_z

exhaustiveness = $exhaustiveness
cpu = $cpus
EOL

done < $pdbs_list

### Accelerated run using QuickVina-W

AcetylCoA ligand is the one from 7dqg

In [ ]:
mkdir -p logs
while read pdb_filename
do
pdb=$(basename -s .pdbqt $pdb_filename)
rm -rf ./runs/$pdb
mkdir -p ./runs/$pdb
echo "Docking $(basename -s .pdbqt $ligand_name) in $(basename -s .pdbqt $pdb)"
qvina-w --config ./configs/$pdb.conf \
        --out ./runs/$pdb/$pdb.docked.pdbqt \
        | tee ./logs/$pdb.log
done < $pdbs_list
echo "DONE"